# PyOCRのインストール

In [ ]:
!pip3 install pyocr

# tesseract-ocrのインストール

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

# tesseract-ocrのVersionを確認

4.0.0-beta.1を前提に説明していきます

In [ ]:
!tesseract -v

# 学習済みモデルのダウンロード

https://github.com/tesseract-ocr/tessdata/blob/4.0.0/ より4.0.0の学習済みモデルをダウンロード

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/blob/4.0.0/jpn.traineddata?raw=true -O jpn.traineddata

# 指定フォルダにコピー

/usr/share/tesseract-ocr/4.00/tessdata/ 以下にコピーします

In [ ]:
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

# PyOCRにtesseract-ocrを取り込み

In [ ]:
import os
from PIL import Image
import pyocr
import pyocr.builders

ocrs = pyocr.get_available_tools()
ocr = ocrs[0]

# Testデータで認識確認

test1.png, test2.png, test3.png をつかってテストデータの認識が問題ないか確認します

In [ ]:
test_data = Image.open("test1.png")
builder = pyocr.builders.TextBuilder(tesseract_layout=3)
result = ocr.image_to_string(test_data, lang="jpn", builder=builder)

print(result)

# Videoカメラの認識確認

下記コマンドでVideoカメラが認識できているか確認します

In [ ]:
!ls /dev/video0

# Videoカメラの取り込み

本サンプルではオートフォーカス対応のカメラが必要です。<br>
奨励カメラ: Logicool C920n, Logicool C615n

In [ ]:
from jetcam.usb_camera import USBCamera

#TODO change capture_device if incorrect for your system
camera = USBCamera(width=640, height=480, capture_width=640, capture_height=480, capture_device=0)

In [ ]:
image = camera.read()

# カメラ画像のNotebookへの表示

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

# カメラ画像をUpdateする処理

In [ ]:
camera.running = True
image = ""
def update_image(change):
    global image
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)

# OCRボタンが押されたら呼び出される処理

In [ ]:
def run_ocr():
    global image
    print("start scanning:")
    ocr_image = Image.frombytes('RGB', (640,480), image)
    builder = pyocr.builders.TextBuilder(tesseract_layout=3)
    result = ocr.image_to_string(ocr_image , lang="jpn", builder=builder)
    print("result")
    print("------------------")
    print(result)
    print("------------------")

# OCRボタンの処理

In [ ]:
ocr_button = ipywidgets.Button(description='ocr')
ocr_button.on_click(lambda c: run_ocr())
tool_widget = ipywidgets.VBox([
    ocr_button
])

# 画像とOCRボタンを表示

結果の表示は、JupyterLabのメニューの

In [ ]:
display(tool_widget)
display(image_widget)
camera.observe(update_image, names='value')

# 終了処理

カメラの更新を停止

In [ ]:
camera.unobserve(update_image, names='value')

# OSの初期化

カメラも初期化されます

In [ ]:
import os
os._exit(00)